In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.python.ops import seq2seq
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np

from tqdm import tqdm

### Load data

In [19]:
from util import load_digits

_, X, Y = load_digits()

sample, timestep, feature = X.shape

### Learning Parameters

In [18]:
# Learning parameters
learning_rate = 1e-3
nb_epochs = 100

batch_size = 64

hidden_dim = 128
latent_dim = 2

In [8]:
# Define Cell
with tf.variable_scope('encoder'):
    en_cell = tf.nn.rnn_cell.GRUCell(hidden_dim)
with tf.variable_scope('decoder'):
    de_cell = tf.nn.rnn_cell.GRUCell(hidden_dim)

In [9]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)

### Encoder

In [20]:
# Input placeholder
x  = tf.placeholder(tf.float32, shape=[None, timestep, feature])

# unpack sequence
encoder_inputs = tf.unpack(tf.transpose(x, perm=[1,0,2]))
encoder_outputs, encoder_state = tf.nn.rnn(en_cell, encoder_inputs, dtype=tf.float32)

last_output = encoder_outputs[-1]

ValueError: Variable RNN/GRUCell/Gates/Linear/Matrix already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-10-79ac6ba1a2a1>", line 6, in <module>
    encoder_outputs, encoder_state = tf.nn.rnn(en_cell, encoder_inputs, dtype=tf.float32)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py", line 2825, in run_ast_nodes
    if self.run_code(code, result):


### Decoder

In [11]:
decoder_inputs = [ last_output ] + \
[ tf.placeholder(tf.float32, shape=[ None, hidden_dim ] ) for i in range( timestep -1 ) ]

def loop(prev, i):
    return prev # Output as input

decoder_outputs, decoder_state = seq2seq.rnn_decoder( decoder_inputs, tf.zeros_like(encoder_state), de_cell, loop_function = loop )

W_out = tf.Variable( xavier_init(hidden_dim, feature) )
b_out = tf.Variable( tf.zeros([ feature ] ) )

unpacked_reconstruction = [ tf.matmul( out, W_out ) for out in decoder_outputs ]

x_ = tf.nn.sigmoid( tf.transpose(tf.pack(unpacked_reconstruction), perm=[1, 0, 2]) )

In [13]:
latent_loss = -0.5 * tf.reduce_sum(1 + z_logvar - tf.square(z_mean) - tf.exp(z_logvar), 1)
reconstruct_loss = tf.reduce_sum(tf.square(x_-x),[1,2])

latent_cost = tf.reduce_mean(latent_loss)
reconstruct_cost = tf.reduce_mean(reconstruct_loss)

cost = latent_cost + reconstruct_cost

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [14]:
saver = tf.train.Saver()

In [15]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    print "Initializing..."
    sess.run(init)

    for epoch in range(nb_epochs):
        print "Epoch",epoch+1
        for _ in tqdm(range( 55000/ batch_size)):
            batch_x, _ = mnist.train.next_batch(batch_size)
            batch_x = batch_x.reshape(batch_size, timestep, feature)
            sess.run(optimizer, feed_dict={x: batch_x })
        X_test = mnist.test.images
        X_test = X_test.reshape(X_test.shape[0], timestep, feature)
        c, lc, rc = sess.run([cost, latent_cost, reconstruct_cost], feed_dict= { x: X_test })
        print "Test set cost {}, latent cost {}, reconstruct cost {}".format(c, lc, rc)

    examples_to_show = 10
    images_to_show = mnist.test.images[:examples_to_show].reshape(examples_to_show,timestep,feature)
    encode_decode = sess.run(x_, feed_dict={x: images_to_show })
    
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    for i in range(examples_to_show):
        a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
        a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
        f.show()
        plt.draw()
    
   
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)


Initializing...


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1


NameError: name 'mnist' is not defined

In [ ]:
print "Illustrating latent dimension"
with tf.Session() as sess:
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    
    examples_to_show = 5000
    x_sample, y_sample = mnist.test.next_batch(examples_to_show)
    x_sample = x_sample.reshape(examples_to_show,timestep,feature)
    z_mu = sess.run( z_logvar, feed_dict={x: x_sample })
    
    plt.figure(figsize=(8,6))
    plt.scatter( z_mu[:, 0], z_mu[:,1], c=np.argmax(y_sample,1) )
    plt.colorbar()

In [ ]:
print "Illustrating latent dimension"
with tf.Session() as sess:
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    
    nx = ny = 20
    x_values = np.linspace(-3, 3, nx)
    y_values = np.linspace(-3, 3, ny)

    canvas = np.empty((28*ny, 28*nx))
    for i, xi in enumerate(x_values):
        for j, yi in enumerate(y_values):
            z_mu = np.array([[xi,yi]])
            x_mean = sess.run( x_, feed_dict={z: z_mu, encoder_state: np.zeros((1,128)) })
            canvas[(nx-i-1)*28:(nx-i)*28, j*28:(j+1)*28] = x_mean[0].reshape(28, 28)

plt.figure(figsize=(8, 10))        
Xi, Yi = np.meshgrid(x_values, y_values)
plt.imshow(canvas, origin="upper")
plt.tight_layout()